In [ ]:
import matplotlib.pyplot as plt

images, labels = next(iter(train_loader))

plt.figure(figsize=(8,8))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(images[i].permute(1,2,0))
    plt.title(class_names[labels[i]])
    plt.axis("off")

plt.show()


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
from sklearn.model_selection import train_test_split

# -------------------------
# CONFIG
# -------------------------
DATASET_PATH = "Dataset" # Ensure this matches your folder name
BATCH_SIZE = 32
IMG_SIZE = 128
LEARNING_RATE = 0.001
EPOCHS = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {device}")

# -------------------------
# AUGMENTATION
# -------------------------
# Training: Heavy augmentation to prevent overfitting
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # Normalization helps CNNs converge faster
])

# Validation: No randomness, just resize and normalize
val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# -------------------------
# DATASET LOADING (The Fix)
# -------------------------
# 1. Load data just to get class names and indices
temp_dataset = datasets.ImageFolder(DATASET_PATH)
targets = temp_dataset.targets
class_names = temp_dataset.classes
print(f"Classes: {class_names}")

# 2. Generate stratified split indices (ensures balanced classes in train/val)
train_idx, val_idx = train_test_split(
    np.arange(len(targets)), 
    test_size=0.2, 
    shuffle=True, 
    stratify=targets, 
    random_state=42
)

# 3. Create two separate dataset objects
train_dataset = datasets.ImageFolder(DATASET_PATH, transform=train_transform)
val_dataset = datasets.ImageFolder(DATASET_PATH, transform=val_transform)

# 4. Create Subsets using the indices
train_ds = Subset(train_dataset, train_idx)
val_ds = Subset(val_dataset, val_idx)

print(f"Train size: {len(train_ds)}")
print(f"Val size: {len(val_ds)}")

# -------------------------
# DATALOADERS
# -------------------------
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print("✅ Data loaders ready")

Using Device: cuda
Classes: ['with_mask', 'without_mask']
Train size: 15476
Val size: 3869
✅ Data loaders ready


In [ ]:
import sys
import os

# go one folder back to main project
sys.path.append("..")
from data_pipeline import get_dataloaders
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

train_loader, val_loader, class_names = get_dataloaders()

print("Classes:", class_names)
print("Train batches:", len(train_loader))
print("Val batches:", len(val_loader))


Using device: cuda
Classes: ['with_mask', 'without_mask']
Train batches: 484
Val batches: 121
